In [ ]:
import batoid
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
telescope = batoid.Optic.fromYaml("LSST_r.yaml")

In [ ]:
def pupil(thx, thy, nside=512):
    rays = batoid.RayVector.asGrid(
        optic=telescope, wavelength=620e-9,
        theta_x=thx, theta_y=thy,
        nx=nside, ny=nside
    )
    rays2 = rays.copy()
    telescope.stopSurface.interact(rays2)
    telescope.trace(rays)
    w = ~rays.vignetted
    return rays2.x[w], rays2.y[w]

In [ ]:
def drawCircle(ax, cx, cy, r, **kwargs):
    t = np.linspace(0, 2*np.pi, 1000)
    x = r*np.cos(t)+cx
    y = r*np.sin(t)+cy
    ax.plot(x, y, **kwargs)

In [ ]:
def drawRay(ax, cx, cy, width, theta, **kwargs):
    R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    
    dx = np.linspace(0, 4.1, 1000)
    dy = np.ones_like(dx)*width/2

    bx = np.copy(dx)
    by = -dy
    
    dx, dy = R.dot(np.vstack([dx, dy]))
    bx, by = R.dot(np.vstack([bx, by]))
    
    dx += cx
    dy += cy
    bx += cx
    by += cy
    
    ax.plot(dx, dy, **kwargs)
    ax.plot(bx, by, **kwargs)

In [ ]:
def drawRectangle(ax, cx, cy, width, height, **kwargs):
    x = width/2*np.array([-1,-1,1,1,-1])
    y = height/2*np.array([-1,1,1,-1,-1])
    x += cx
    y += cy
    ax.plot(x, y, **kwargs)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def modelPlot(thx, thy):
    fig, ax = plt.subplots(1, 1, figsize=(30, 30))
    ax.scatter(*pupil(thx,thy), s=0.1, c='k')
    ax.set_aspect('equal')
    # Primary mirror
    drawCircle(ax, 0, 0, 4.18, c='r')
    drawCircle(ax, 0, 0, 2.55, c='r')
    
    # Interior vignetting
    drawCircle(ax, -17*thx, -17*thy, 2.35, c='r')
    
    # First exterior vignetting
    drawCircle(ax, -33*thx, -33*thy, 4.9, c='r')

    # Second exterior vignetting    
    thr = np.hypot(thx, thy)
    if thr > np.deg2rad(1.5):
        thph = np.arctan2(thy, thx)
        cr = -40.68 + 2365*thr - 49274*thr**2
        drawCircle(ax, cr*np.cos(thph), cr*np.sin(thph), 18.0, c='g')

    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    ax.axvline(c='k')
    ax.axhline(c='k')
    fig.show()

In [ ]:
modelPlot(np.deg2rad(1.76), np.deg2rad(0.5))